<h1>
<center>Empirical Asset Pricing via Machine Learning</center>
</h1>



<center>Alexander Margetis, Lanya Ma, Sheng Yang, Yiming Tan</center>



## 1. Introduction



In this project, we conducted an empirical analysis of machine learning methods in asset pricing. We aim to measure and predict risk premium with bundles of underlying factors. In particular, we attempt to study the structure of cross-sectional returns by using various factors. These factors can be stock-level firm characteristics, macroeconomic descriptors and many other derived indicators. In the empirical literature, classical models are proposed to estimate and explain the risk premia with several factors, like CAPM, Fama-French 3 factor and later Fama-French 5 factor model. These models are basically linear projection from bahavior of stocks' expected returns to multiple variates. As the high-dimensional nature is innate in machine learning methods, we can enhances the flexibility of representing assets risk profile relative to more traditional econometric prediction techniques.And the functionals which project high-dimensional predictors to risk premia can be complicated. That's why the application of machine learning in this field can be rather attractive.   

Our major contributions in this project are three-fold. First, we investigate machine learning techniques in prediction of cross-sectional returns. This tells us whether machine learning algorithms can improve the estimation of out-of-sample expected returns. Second, we examine the feature importance of factors.This process gives us insights that how to select informative factors. Third, we study the stability of machine learning in portfolio constrcution. We analyze the performance of long-short portfolio from the algorithms over the horizon.

## 2. Methodology

### 2.1 Data Preprocessing and Exploratory Data Analysis (Lanya Ma and Yiming Tan)

Data Cleaning (Lanya Ma)

In [1]:
import pandas as pd
import os
import glob
import numpy as np

os.chdir(os.getcwd() + '\\data')
extension = 'csv'
result = glob.glob('*.{}'.format(extension))
print(result)
data_name_list = list(map(lambda x: x[:-4] , result)) 

['3m_tbill_rate.csv', 'bps.csv', 'ca_to_assets.csv', 'current.csv', 'debt_over_asset.csv', 'ebitda_to_sales.csv', 'eps_ttm.csv', 'equity_to_totalcap.csv', 'GICS_code.csv', 'market_cap.csv', 'netprofit_margin.csv', 'ocf_ps.csv', 'ocf_to_debt.csv', 'ocf_to_sales.csv', 'op_to_debt.csv', 'pb.csv', 'pe_ttm.csv', 'price.csv', 'ps_ttm.csv', 'roa.csv', 'roe.csv', 'roic.csv', 'sales_growth1yr.csv', 'tax_to_ebt.csv', 'turnover.csv']


In [2]:
import copy
import numpy as np

def transform_row(filename):
    raw_df = pd.read_csv(filename)
    row_names = raw_df.iloc[:,0]
    raw_df = raw_df.rename(index = row_names)
    new_df = raw_df.drop(raw_df.columns[0],axis = 1)
    return new_df

#Omit stocks w/ no price
price = transform_row('price.csv')
sum_df = pd.DataFrame(price.sum(axis = 1)).reset_index()
valid = sum_df[sum_df[0]!=0].index.values

price_v = price.iloc[valid,:]
is_available = copy.deepcopy(price_v)
is_available[:] = np.nan
is_available[price_v != 0] = 1
price_df = price_v*is_available


def tranfrom_missing(filename):
    raw =  transform_row(filename)
    raw = raw.iloc[valid,:]
    nrow = raw.shape[0]
    ncol = raw.shape[1]
    raw = raw.applymap(lambda x: float(x))
    raw_mat = np.asmatrix(raw)
    for i in range(0,nrow):
        fill_data = 0
        for j in range(0,ncol):
            if raw_mat[i,j] != 0:
                fill_data = raw_mat[i,j]
            else:
                raw_mat[i,j] = fill_data
    filled_df = pd.DataFrame(raw_mat)
    filled_df.index = price_v.index.values
    filled_df.columns = price_v.columns.values
    filled_df = np.multiply(filled_df,is_available)
    filled_df[filled_df == 0] = np.nan
    return filled_df


sector_code = transform_row('GICS_code.csv')
sector_code[sector_code ==0] = np.nan
"""
sector_intcode = []
for code in sector_code['GICS_Code']:
    if ~np.isnan(code):
        code = int(code)
    sector_intcode.append(code)
"""

# Consider S&P Dow Jones Indices and MSCI changed sector classification
sector_dict = {1010:"Energy", 1510:"Materials",2010:"Capital Goods", 2020:"Commercial & Professional Services",
               2030:"Transportation", 2510:"Automobiles & Components", 2520:"Consumer Durables & Apparel",
               2530:"Consumer Services", 2540:"Media & Entertainment", 2550:"Retailing", 3010:"Food & Staples Retailing",
               3020:"Food, Beverage & Tobacco", 3030:"Household & Personal Products", 3510:"Health Care Equipment & Services",
               3520:"Pharmaceuticals, Biotechnology & Life Sciences", 4010:"Banks", 4020:"Diversified Financials",
               4030:"Insurance", 4040:"Real Estate",4510:"Software & Services", 4520:"Technology Hardware & Equipment",
               4530:"Semiconductors & Semiconductor Equipment", 5010:"Telecommunication Services", 5020:"Media & Entertainment",
               5510:"Utilities", 6010:"Real Estate"}    

sectors_list = []
for code in sector_code['GICS_Code']:
    if ~np.isnan(code):
        code = sector_dict[code]
    sectors_list.append(code)

In [3]:
features_dict = {}
numeric_features = []
for idx, element in enumerate(result):
    feature_name = data_name_list[idx]
    if feature_name not in ['3m_tbill_rate','GICS_code','price']:
        feature_df = tranfrom_missing(element)
        features_dict[data_name_list[idx]] = feature_df
        numeric_features.append(feature_name)

In [4]:
#Stock return
return_mon = (price_df.T.shift(1).T - price_df)/price_df
# shift the next month return ahead so that we can concatenate features and target
return_mon = return_mon.T.shift(-1).T
#Risk free rate
rf= transform_row('3m_tbill_rate.csv')
rfree = rf['rate'][return_mon.columns.values]/12

In [5]:
#Raw Data Dictionary
data_dict = {}
time_keys = price.columns.values
first_feature = list(features_dict.keys())[0]
sectors_array = np.array(sectors_list).reshape(-1, 1)
sectors_array = sectors_array[valid]
for time_key in time_keys[:-1]:
    cat_array = features_dict[first_feature][time_key].values.reshape(-1, 1)
    for key in list(features_dict.keys())[1:]:
        right_array = features_dict[key][time_key].values.reshape(-1, 1)
        cat_array = np.concatenate((cat_array,right_array),axis = 1)
    y_array = return_mon[time_key].values.reshape(-1,1)-rfree[time_key]
    cat_array = np.concatenate((cat_array,sectors_array),axis = 1)
    data_array = np.concatenate((cat_array,y_array),axis = 1)
    data_df = pd.DataFrame(data_array)
    data_df.index = price_df.index.values
    data_df.columns = list(features_dict.keys()) + ['Sectors'] + ['Excess_return']
    data_dict[time_key] = data_df

In [6]:
import pickle
with open('factordata.p', 'wb') as fp:
    pickle.dump(data_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
import pickle
with open('factordata.p', 'rb') as fp:
    factordata = pickle.load(fp)

Build data pipeline(Yiming Tan)

In [8]:
def valid_return(dataframe):
    # will raise error with object type in nparray
    
    nrow = data_df.shape[0]
    for i in range(0,nrow):
        if not np.isnan(dataframe['Excess_return'][i]):
            exist.append(i)
    dat = dataframe.iloc[exist,:]
    return dat

def valid_returnV2(dataframe):
    dataframe['Excess_return'] = dataframe['Excess_return'].astype(float)
    # exclude rows where Excess_return is nan
    dat = dataframe[np.isfinite(dataframe['Excess_return'])]
    return dat

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import Imputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import warnings
import sys
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

num_pipeline = Pipeline([
        ('imp', IterativeImputer(max_iter=10, random_state=0,estimator = KNeighborsRegressor(n_neighbors=15))),
        ('std_scaler', StandardScaler()),
    ])
cat_pipeline = Pipeline([
        ('imputer', Imputer(missing_values='NaN', strategy='most_frequent', axis=0)),
        ('one', OneHotEncoder(categories='auto')),
    ])


X = {}
y = {}

In [9]:
# fill missing sector with most_frequent item
cat_counts = np.unique(sectors_array, return_counts=True)
most_freq_sec = cat_counts[0][np.argmax(cat_counts[1])]
sectors_array[sectors_array == 'nan'] = most_freq_sec
set(sectors_array.ravel())

{'Automobiles & Components',
 'Banks',
 'Capital Goods',
 'Commercial & Professional Services',
 'Consumer Durables & Apparel',
 'Consumer Services',
 'Diversified Financials',
 'Energy',
 'Food & Staples Retailing',
 'Food, Beverage & Tobacco',
 'Health Care Equipment & Services',
 'Household & Personal Products',
 'Insurance',
 'Materials',
 'Media & Entertainment',
 'Pharmaceuticals, Biotechnology & Life Sciences',
 'Real Estate',
 'Retailing',
 'Semiconductors & Semiconductor Equipment',
 'Software & Services',
 'Technology Hardware & Equipment',
 'Telecommunication Services',
 'Transportation',
 'Utilities'}

In [10]:
onehotencoder = OneHotEncoder(categories='auto')
encoded_array = onehotencoder.fit_transform(sectors_array.reshape(-1,1)).toarray()
dummy_names = onehotencoder.get_feature_names().astype(str)
dummy_names = list(map(lambda x: x[3:] , dummy_names))

In [11]:
for time_key in time_keys[:-1]:
    raw_df = data_dict[time_key].drop('Sectors', axis=1)
    complete_array = np.concatenate((raw_df,encoded_array),axis=1)
    complete_df = pd.DataFrame(complete_array)
    whole_col_names = numeric_features + ['Excess_return'] + dummy_names
    complete_df.columns = whole_col_names
    dat = valid_returnV2(complete_df)
    num_features = dat.drop('Excess_return',axis=1).iloc[:,:len(numeric_features)]
    cat_features = dat.iloc[:, len(numeric_features)+1:]
    num_features_ = num_pipeline.fit_transform(num_features)
    X_array = np.concatenate((num_features_,cat_features),axis=1)
    X[time_key] = pd.DataFrame(X_array)
    X[time_key].index = dat.index.values
    X[time_key].columns = numeric_features + dummy_names
    y[time_key] = pd.DataFrame(dat['Excess_return'])
    y[time_key].index = dat.index.values

In [12]:
Train = {}
Test = {}
for time_key in time_keys[:100]:
    Train[time_key] = pd.concat([X[time_key],y[time_key]],axis=1)
for time_key in time_keys[100:-1]:
    Test[time_key] = pd.concat([X[time_key],y[time_key]],axis=1)

In [13]:
import pickle
with open('cleanTrain.p', 'wb') as gp:
    pickle.dump(Train, gp, protocol=pickle.HIGHEST_PROTOCOL)
with open('cleanTest.p', 'wb') as hp:
    pickle.dump(Test, hp, protocol=pickle.HIGHEST_PROTOCOL)

### 2.2 Generalized linear models


### 2.3 Support vector machines 


### 2.4 Ensemble Learning

### 2.5 Neural Networks

In [17]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6914345501951685237
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4951913267
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3575991285628681239
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


### 2.6 Dimension Reduction 

## 3. Experimental Results

### 3.1 Data Description and Exploratory Data Analysis




Our dataset in this project includes all listed firms in the NYSE, AMEX, and NASDAQ. Our sample begins in January 2009 to April 2019. Our data is monthly updated. We use the Treasury-bill rate as risk-free rate for calculating the excessive returns. The firms characteristics or the factors in other words, include firms' value, growth, solvency, cash flow, profitability, operating capacity, capital structure and momentum. In addition, we include the categorical industry classes corresponding to GICS sectors.    

### 3.2 Out-of-sample Stock-level Prediction Performance



### 3.3 Variable Importance for factors

### 3.4 Machine Learning Portfolios

## 4. Conclusions

## 5. References

Gu, Shihao, Bryan Kelly, and Dacheng Xiu. *Empirical asset pricing via machine learning.* No. w25398. National Bureau of Economic Research, 2018.

